In [14]:
import sys, os
import pandas as pd
import numpy as np

from adabound import AdaBound

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [2]:
df=pd.read_csv('fer2013.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [4]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
X_train,train_y,X_test,test_y=[],[],[],[]

In [6]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [7]:
num_features = 64
num_labels = 7
batch_size = 100
epochs = 50
width, height = 48, 48

In [8]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [9]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [10]:
#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [12]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.1))

# #4th convolution layer
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0         
__________

In [15]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=AdaBound(),
              metrics=['accuracy'])

In [16]:
#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28709 samples, validate on 3589 samples
Epoch 1/50




28709/28709 [==============================] - 25s 863us/step - loss: 1.7838 - acc: 0.2559 - val_loss: 1.7306 - val_acc: 0.2898
Epoch 2/50
28709/28709 [==============================] - 20s 694us/step - loss: 1.6569 - acc: 0.3383 - val_loss: 1.5689 - val_acc: 0.3828
Epoch 3/50
28709/28709 [==============================] - 19s 650us/step - loss: 1.5426 - acc: 0.3909 - val_loss: 1.4464 - val_acc: 0.4480
Epoch 4/50
28709/28709 [==============================] - 18s 642us/step - loss: 1.4576 - acc: 0.4299 - val_loss: 1.3815 - val_acc: 0.4620TA: 0s - loss: 1.4591
Epoch 5/50
28709/28709 [==============================] - 18s 640us/step - loss: 1.4051 - acc: 0.4559 - val_loss: 1.3361 - val_acc: 0.4826
Epoch 6/50
28709/28709 [==============================] - 18s 634us/step - loss: 1.3562 - acc: 0.4793 - val_loss: 1.3139 - val_acc: 0.4

Epoch 42/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7522 - acc: 0.7136 - val_loss: 1.1635 - val_acc: 0.5965
Epoch 43/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7423 - acc: 0.7206 - val_loss: 1.1614 - val_acc: 0.5974
Epoch 44/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7285 - acc: 0.7267 - val_loss: 1.1595 - val_acc: 0.6013
Epoch 45/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7231 - acc: 0.7290 - val_loss: 1.1762 - val_acc: 0.5896
Epoch 46/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7112 - acc: 0.7321 - val_loss: 1.1749 - val_acc: 0.5954
Epoch 47/50
28709/28709 [==============================] - 17s 580us/step - loss: 0.7007 - acc: 0.7342 - val_loss: 1.1897 - val_acc: 0.5952
Epoch 48/50
28709/28709 [==============================] - 17s 579us/step - loss: 0.6820 - acc: 0.7420 - val_loss: 1.1902 - val_acc: 0.6066
Epoch 49/50
28709/28

In [17]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [ ]:
import sklearn

sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)